In [ ]:
import torch
from controlnet_aux import CannyDetector
from diffusers import FluxControlPipeline
from diffusers.utils import load_image

In [ ]:
pipe = FluxControlPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16).to("cuda")
pipe.load_lora_weights("black-forest-labs/FLUX.1-Canny-dev-lora", adapter_name="canny")
pipe.set_adapters("canny", 0.85)

In [ ]:
def resize_image(image, )

In [ ]:
prompt = "A robot made of exotic candies and chocolates of different kinds. The background is filled with confetti and celebratory gifts."
control_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/robot.png")

processor = CannyDetector()
control_image = processor(control_image, low_threshold=50, high_threshold=200, detect_resolution=1024, image_resolution=1024)

image = pipe(
    prompt=prompt,
    control_image=control_image,
    height=1024,
    width=1024,
    num_inference_steps=50,
    guidance_scale=30.0,
).images[0]
image

In [ ]:
import torch
from diffusers import FluxFillPipeline
from diffusers.utils import load_image

image = load_image("https://huggingface.co/datasets/diffusers/diffusers-images-docs/resolve/main/cup.png")
mask = load_image("https://huggingface.co/datasets/diffusers/diffusers-images-docs/resolve/main/cup_mask.png")

pipe = FluxFillPipeline.from_pretrained("black-forest-labs/FLUX.1-Fill-dev", torch_dtype=torch.bfloat16).to("cuda")

In [ ]:

image = pipe(
    prompt="a white paper cup",
    image=image,
    mask_image=mask,
    height=1632,
    width=1232,
    guidance_scale=30,
    num_inference_steps=10,
    max_sequence_length=512,
    generator=torch.Generator("cpu").manual_seed(0)
).images[0]
image.save(f"flux-fill-dev.png")

In [127]:
def resize_image(image, scale_percent=30):
    """Resize image by percentage"""
    width = int(image.size[0] * (1 - scale_percent/100))
    height = int(image.size[1] * (1 - scale_percent/100))
    return image.resize((width, height), Image.Resampling.LANCZOS)

def create_image_with_mask(image, target_size=(1024, 1024), mask_percent=30):
    """Create black background with image and mask
    Args:
        image: Input image
        target_size: Size of output image (width, height)
        mask_percent: Percentage of target image that mask will cover
    """
    # Create black background
    background = Image.new('RGB', target_size, (0, 0, 0))
    
    # Calculate position to center the image
    x = (target_size[0] - image.size[0]) // 2
    y = (target_size[1] - image.size[1]) // 2
    
    # Calculate mask size based on percentage
    mask_width = int(target_size[0] * (mask_percent/100))
    mask_height = int(target_size[1] * (mask_percent/100))
    
    # Create mask (white where image will be, black for background)
    mask = Image.new('L', target_size, 255)
    
    # Calculate mask position to center it
    mask_x = (target_size[0] - mask_width) // 2
    mask_y = (target_size[1] - mask_height) // 2
    
    # Create black rectangle in mask
    mask.paste(0, (mask_x, mask_y, mask_x + mask_width, mask_y + mask_height))
    
    # Calculate crop box to get center of image
    left = (image.size[0] - mask_width) // 2
    top = (image.size[1] - mask_height) // 2
    right = left + mask_width
    bottom = top + mask_height
    
    # Crop image from center to mask size
    cropped_image = image.crop((left, top, right, bottom))
    
    # Paste cropped image onto black background at mask position
    background.paste(cropped_image, (mask_x, mask_y))
    
    return background, mask

In [128]:
from PIL import Image

In [ ]:
# 1. Load your image
image = Image.open('/root/lo.png')

# 2. Resize image by 30% smaller
resized_image = resize_image(image, scale_percent=5)

# 3. Create black background with image and mask
final_image, mask = create_image_with_mask(resized_image, target_size=(1024, 1024))
final_image.resize((500, 500))

In [ ]:
mask.resize((500, 500))

In [ ]:
# 4. Use with pipeline
result = pipe(
    prompt="A majestic peacock with a dazzling golden tail fully fanned out, featuring gemstone-like red accents on each feather. The peacock has a rich teal and emerald green body, elegant golden and purple wings, and a regal crown of feathers atop its head. It is surrounded by blooming pink and magenta flowers, with glowing petals falling around it. The background is a luxurious deep purple with golden highlights, creating a royal and magical atmosphere. Digital art, fantasy style, ultra-detailed, vibrant colors.",
    image=final_image,
    mask_image=mask,
    height=512,
    width=512,
    guidance_scale=30,
    num_inference_steps=60,
    max_sequence_length=512,
    generator=torch.Generator("cpu").manual_seed(0)
).images[0]
result.save("result.png")

In [ ]:
result

In [ ]:
mask

In [ ]:
image

In [ ]:
import torch
from controlnet_aux import CannyDetector
from diffusers import FluxControlPipeline
from diffusers.utils import load_image

pipe = FluxControlPipeline.from_pretrained("black-forest-labs/FLUX.1-Canny-dev", torch_dtype=torch.bfloat16).to("cuda")

In [ ]:
print('hi')

In [ ]:
image_path = '/root/round.jpg'

prompt = "A robot made of exotic candies and chocolates of different kinds. The background is filled with confetti and celebratory gifts."
control_image = load_image(image_path)

processor = CannyDetector()
control_image = processor(control_image, low_threshold=50, high_threshold=200, detect_resolution=1024, image_resolution=1024)

image = pipe(
    prompt=prompt,
    control_image=control_image,
    controlnet_conditioning_scale=1.0,
    height=1024,
    width=1024,
    num_inference_steps=50,
    guidance_scale=30.0,
).images[0]
image.save("output.png")

In [ ]:
print('hi')

In [ ]:
import torch
from diffusers.utils import load_image
from diffusers.pipelines.flux.pipeline_flux_controlnet import FluxControlNetPipeline
from diffusers.models.controlnet_flux import FluxControlNetModel

base_model = 'black-forest-labs/FLUX.1-dev'
controlnet_model = 'InstantX/FLUX.1-dev-Controlnet-Canny'
controlnet = FluxControlNetModel.from_pretrained(controlnet_model, torch_dtype=torch.bfloat16)
pipe = FluxControlNetPipeline.from_pretrained(base_model, controlnet=controlnet, torch_dtype=torch.bfloat16)
pipe.to("cuda")

In [6]:
from PIL import Image

In [7]:
def extend_transparent_background(image, scale=2):
    # Open the image
    img = image.convert("RGB")
    width, height = img.size

    # Calculate new size (scale > 1 will make the image canvas larger)
    new_width = int(width * scale)
    new_height = int(height * scale)

    # Create a new white image with the new size
    new_img = Image.new("RGB", (new_width, new_height), (255, 255, 255))

    # Paste the original image in the center
    offset = ((new_width - width) // 2, (new_height - height) // 2)
    new_img.paste(img, offset)
    return new_img

In [24]:
control_image = load_image('/root/round_white.png')

In [ ]:
control_image

In [ ]:


control_image = load_image('/root/round_white.png')
control_image = extend_transparent_background(control_image, 2)
prompt = "A majestic, roaring lion with a vibrant, fiery orange mane, sharp teeth, and intense golden eyes. The lion is facing left with its mouth open, exuding power and authority. The background is a warm gradient of red and orange, enhancing the dramatic and regal feel. The style is hyper-realistic with a touch of digital illustration, emphasizing bold lighting and vivid colors."
image = pipe(
    prompt, 
    control_image=control_image,
    controlnet_conditioning_scale=0.2,
    num_inference_steps=28, 
    guidance_scale=3.5,
).images[0]

In [ ]:
image.resize((500, 500))

In [ ]:


control_image = load_image('/root/round_white.png')
control_image = extend_transparent_background(control_image, 1)
prompt = "A majestic, roaring lion with a vibrant, fiery orange mane, sharp teeth, and intense golden eyes. The lion is facing left with its mouth open, exuding power and authority. The background is a warm gradient of red and orange, enhancing the dramatic and regal feel. The style is hyper-realistic with a touch of digital illustration, emphasizing bold lighting and vivid colors."
image = pipe(
    prompt, 
    control_image=control_image,
    controlnet_conditioning_scale=0.2,
    num_inference_steps=28, 
    guidance_scale=3.5,
).images[0]

In [ ]:
image.resize((500, 500))

In [ ]:


control_image = load_image('/root/round_white.png')
control_image = extend_transparent_background(control_image, 4)
prompt = "A majestic, roaring lion with a vibrant, fiery orange mane, sharp teeth, and intense golden eyes. The lion is facing left with its mouth open, exuding power and authority. The background is a warm gradient of red and orange, enhancing the dramatic and regal feel. The style is hyper-realistic with a touch of digital illustration, emphasizing bold lighting and vivid colors."
image = pipe(
    prompt, 
    control_image=control_image,
    controlnet_conditioning_scale=0.2,
    num_inference_steps=28, 
    guidance_scale=3.5,
).images[0]
image.resize((500, 500))

In [ ]:


control_image = load_image('/root/round_white.png')
control_image = extend_transparent_background(control_image, 5)
prompt = "A majestic, roaring lion with a vibrant, fiery orange mane, sharp teeth, and intense golden eyes. The lion is facing left with its mouth open, exuding power and authority. The background is a warm gradient of red and orange, enhancing the dramatic and regal feel. The style is hyper-realistic with a touch of digital illustration, emphasizing bold lighting and vivid colors."
image = pipe(
    prompt, 
    control_image=control_image,
    controlnet_conditioning_scale=0.2,
    num_inference_steps=28, 
    guidance_scale=3.5,
).images[0]
image.resize((500, 500))

In [ ]:


control_image = load_image('/root/round_white.png')
control_image = extend_transparent_background(control_image, 8)
prompt = "A majestic, roaring lion with a vibrant, fiery orange mane, sharp teeth, and intense golden eyes. The lion is facing left with its mouth open, exuding power and authority. The background is a warm gradient of red and orange, enhancing the dramatic and regal feel. The style is hyper-realistic with a touch of digital illustration, emphasizing bold lighting and vivid colors."
image = pipe(
    prompt, 
    control_image=control_image,
    controlnet_conditioning_scale=0.2,
    num_inference_steps=28, 
    guidance_scale=3.5,
).images[0]
image.resize((500, 500))

In [ ]:
image.resize((500, 500))

In [ ]:
image.resize((500, 500))

In [ ]:
import torch
from diffusers.utils import load_image
from diffusers import FluxControlNetPipeline, FluxControlNetModel

base_model = "black-forest-labs/FLUX.1-dev"
controlnet_model = "Shakker-Labs/FLUX.1-dev-ControlNet-Depth"



In [2]:
controlnet = FluxControlNetModel.from_pretrained(controlnet_model, torch_dtype=torch.bfloat16)

In [ ]:

pipe = FluxControlNetPipeline.from_pretrained(
    base_model, controlnet=controlnet, torch_dtype=torch.bfloat16
)
pipe.to("cuda")



In [ ]:
control_image = load_image('/root/round_white.png')
control_image = extend_transparent_background(control_image, 1)


prompt = "A majestic peacock with a dazzling golden tail fully fanned out, featuring gemstone-like red accents on each feather. The peacock has a rich teal and emerald green body, elegant golden and purple wings, and a regal crown of feathers atop its head. It is surrounded by blooming pink and magenta flowers, with glowing petals falling around it. The background is a luxurious deep purple with golden highlights, creating a royal and magical atmosphere. Digital art, fantasy style, ultra-detailed, vibrant colors."

image = pipe(prompt,
             control_image=control_image,
             controlnet_conditioning_scale=0.2,
             width=1024,
             height=1024,
             num_inference_steps=24,
             guidance_scale=3.5,
).images[0]

In [ ]:
image.resize((500, 500))

In [ ]:
control_image = load_image('/root/round_white.png')
control_image = extend_transparent_background(control_image, 2)


prompt = "A magical glowing book open on a soft, dreamy surface, surrounded by sparkling particles and golden light. The pages emit a warm, enchanting glow with ornate, ancient-style text and decorative borders. Tiny glowing fairies or fireflies hover around the book, enhancing the sense of wonder. The background features a mystical forest with soft-focus trees and a misty, twilight atmosphere. Fantasy art style, richly detailed, magical and whimsical mood."

image = pipe(prompt,
             control_image=control_image,
             controlnet_conditioning_scale=0.4,
             width=1024,
             height=1024,
             num_inference_steps=24,
             guidance_scale=3.5,
).images[0]

In [ ]:
image.resize((500, 500))

In [ ]:
control_image = load_image('/root/round_white.png')
control_image = extend_transparent_background(control_image, 3)


prompt = "A magical glowing book open on a soft, dreamy surface, surrounded by sparkling particles and golden light. The pages emit a warm, enchanting glow with ornate, ancient-style text and decorative borders. Tiny glowing fairies or fireflies hover around the book, enhancing the sense of wonder. The background features a mystical forest with soft-focus trees and a misty, twilight atmosphere. Fantasy art style, richly detailed, magical and whimsical mood."

image = pipe(prompt,
             control_image=control_image,
             controlnet_conditioning_scale=0.3,
             width=1024,
             height=1024,
             num_inference_steps=24,
             guidance_scale=3.5,
).images[0]
image.resize((500, 500))

In [ ]:
control_image = load_image('/root/round_white.png')
control_image = extend_transparent_background(control_image, 6)


prompt = "A magical glowing book open on a soft, dreamy surface, surrounded by sparkling particles and golden light. The pages emit a warm, enchanting glow with ornate, ancient-style text and decorative borders. Tiny glowing fairies or fireflies hover around the book, enhancing the sense of wonder. The background features a mystical forest with soft-focus trees and a misty, twilight atmosphere. Fantasy art style, richly detailed, magical and whimsical mood."

image = pipe(prompt,
             control_image=control_image,
             controlnet_conditioning_scale=0.3,
             width=1024,
             height=1024,
             num_inference_steps=24,
             guidance_scale=3.5,
).images[0]
image.resize((500, 500))

In [ ]:
control_image